In [25]:
import numpy as np
import polars as pl
import sklearn


In [26]:
df = pl.read_parquet('train_test_merged_df.parquet')
target_feature = pl.read_parquet('target_series.parquet')


In [27]:
df.shape, target_feature.shape

((2919, 78), (1460, 1))

In [28]:
train_df = df.limit(target_feature.height)
test_df = df.slice(target_feature.height)
train_df.shape, test_df.shape

((1460, 78), (1459, 78))

In [29]:
model = sklearn.ensemble.HistGradientBoostingRegressor(
    loss="squared_error",
    learning_rate=0.1,
    max_iter=200,
    max_leaf_nodes=None,
    max_depth=None,
    max_features=0.7,
    categorical_features="from_dtype",
    min_samples_leaf=10,
    l2_regularization=2.0,
    warm_start=True,
    scoring="neg_root_mean_squared_error",
    validation_fraction=0.3,
    verbose=1,
    random_state=42,
)
model.fit(train_df, target_feature)
score = model.score(train_df, target_feature)
print(score)


Binning 0.001 GB of training data: 

/home/ali/Public/Apps/anaconda3/envs/tf-env-0019/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.030 s
Fitting gradient boosted rounds:
Fit 200 trees in 2.844 s, (22139 total leaves)
Time spent computing histograms: 0.738s
Time spent finding best splits:  0.697s
Time spent applying splits:      0.481s
Time spent predicting:           0.019s
0.9989330045564039


In [30]:
preds = model.predict(test_df)


In [31]:
pl.read_csv('sample_submission.csv')

Id,SalePrice
i64,f64
1461,169277.052498
1462,187758.393989
1463,183583.68357
1464,179317.477511
1465,150730.079977
…,…
2915,167081.220949
2916,164788.778231
2917,219222.4234


In [32]:
submission_df = pl.DataFrame([np.arange(1461,2920),preds],schema=["Id","SalePrice"])


In [33]:
submission_df.write_csv('submission.csv')

In [34]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f submission.csv -m "house-prices-advanced-regression-techniques submission #1"

100%|██████████████████████████████████████| 33.6k/33.6k [00:01<00:00, 29.1kB/s]
Successfully submitted to House Prices - Advanced Regression Techniques